In [2]:
import numpy as np 
import pandas as pd 
import altair as alt 
import os 
import re 
import ntpath
from Commons.data_processing import *
from Commons.DataProcessors.pd_processor import PDProcessor


In [11]:
files = get_files(r".\N_Glycosylation_Results")

In [12]:
read_data = None

for file in files:

    base_name, _ = ntpath.splitext(ntpath.basename(file))
    div = base_name.split("-")[0]
    div = div.split("_")
    conc, temp, run = div[-3:]

    data = PDProcessor([file], sample_name="_".join(div[-3:]))
    data.add_special_column("concentration", conc)
    data.add_special_column("temperature", temp)
    data.add_special_column("run", run)

    if read_data is None:
        read_data = data
    else:
        read_data.join_processors(data)


read_data.alias_engine("")


In [13]:
peptides = read_data.peptides[~read_data.peptides.glycan_composition.isna()]

In [20]:
conc_map = {"1": 1.0, "05": 0.5, "025": 0.25, "0125": 0.125, "00625": 0.0625}
peptides.loc[:, 'concentration'] = peptides.concentration.map(conc_map)


C:\Users\Graham Delafield\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-20-d60791fe0344>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  peptides.loc[:, 'concentration'] = peptides.concentration.map(conc_map)


In [21]:
peptides

,accession,description,checked,confidence,annotated_sequence,modifications,num_protein_groups,num_proteins,num_psms,master_protein_accessions,...,q_value_1d,fdr_1d,peptide_group_fdr_1d,sequence_in_protein,positions_in_proteins,sequence,data_source,concentration,temperature,run
2,P61823,Ribonuclease pancreatic OS=Bos taurus OX=9913 ...,True,Medium,[R].NLTK.[D],1xHexNAc(2) [N1],1,1,1,P61823,...,0.0373692,0.037196,0.115151,R.NLTK.D,[60-63],NLTK,00625_30C_Run1,0.0625,30C,Run1
3,P61823,Ribonuclease pancreatic OS=Bos taurus OX=9913 ...,True,High,[R].NLTK.[D],1xHexNAc(2)Hex(1) [N1],1,1,4,P61823,...,1.4599e-05,0,0,R.NLTK.D,[60-63],NLTK,00625_30C_Run1,0.0625,30C,Run1
4,P61823,Ribonuclease pancreatic OS=Bos taurus OX=9913 ...,True,High,[R].NLTK.[D],1xHexNAc(2)Hex(2) [N1],1,1,4,P61823,...,2.3116e-05,0,0,R.NLTK.D,[60-63],NLTK,00625_30C_Run1,0.0625,30C,Run1
5,P61823,Ribonuclease pancreatic OS=Bos taurus OX=9913 ...,True,High,[R].NLTK.[D],1xHexNAc(2)Hex(3) [N1],1,1,5,P61823,...,3.62703e-05,0,0,R.NLTK.D,[60-63],NLTK,00625_30C_Run1,0.0625,30C,Run1
6,P61823,Ribonuclease pancreatic OS=Bos taurus OX=9913 ...,True,High,[R].NLTK.[D],1xHexNAc(2)Hex(4) [N1],1,1,7,P61823,...,1.93477e-05,0,0,R.NLTK.D,[60-63],NLTK,00625_30C_Run1,0.0625,30C,Run1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,P61823,Ribonuclease pancreatic OS=Bos taurus OX=9913 ...,True,Medium,[K].SRNLTKDR.[C],1xHexNAc(6)Hex(3)Fuc(1)NeuAc(2) [N3],1,1,1,P61823,...,0.034626,0.0345781,0.106109,K.SRNLTKDR.C,[58-65],SRNLTKDR,1_60C_Run3,1,60C,Run3
446,P61823,Ribonuclease pancreatic OS=Bos taurus OX=9913 ...,True,High,[K].SRNLTKDR.[C],1xHexNAc(6)Hex(4)NeuAc(2) [N3],1,1,1,P61823,...,0.00378429,0.00367309,0.0195122,K.SRNLTKDR.C,[58-65],SRNLTKDR,1_60C_Run3,1,60C,Run3
447,P61823,Ribonuclease pancreatic OS=Bos taurus OX=9913 ...,True,Medium,[K].SRNLTKDR.[C],1xAcetyl [N-Term]; 1xHexNAc(6)Hex(3)Fuc(1)NeuA...,1,1,1,P61823,...,0.0355921,0.0352065,0.107362,K.SRNLTKDR.C,[58-65],SRNLTKDR,1_60C_Run3,1,60C,Run3
448,P61823,Ribonuclease pancreatic OS=Bos taurus OX=9913 ...,True,High,[K].SRNLTKDR.[C],1xAcetyl [N-Term]; 1xHexNAc(4)Hex(5)NeuAc(3) [N3],1,1,1,P61823,...,0.000751193,0,0,K.SRNLTKDR.C,[58-65],SRNLTKDR,1_60C_Run3,1,60C,Run3


In [22]:
cutoffs = [50, 100, 150, 200, 250, 300, 350, 400, 500]
nums = []
for cut in cutoffs:
    nums.append(len(peptides[peptides.byonic_score >= cut]))

res = pd.DataFrame({"cutoffs": cutoffs, "vals": nums})

alt.Chart(res).mark_bar().encode(x="cutoffs:N", y="vals:Q")


alt.Chart(...)

In [23]:
peptides = peptides[peptides.byonic_score >= 200]
peptides['glycan'] = peptides.glycan_composition

<ipython-input-23-ad9be6f75ba4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  peptides['glycan'] = peptides.glycan_composition


In [58]:
def categorize_glycan(glycan):

    glycan = glycan.replace(")", ",")
    glycan = glycan.replace("(", " ")
    glycan = glycan.split(",")[:-1]
    d = {k: int(v) for [k, v] in [i.split(" ") for i in glycan]}

    if "NeuAc" in d or "NeuGc" in d:
        return "Sialylated"
    elif "Fuc" in d:
        if d["HexNAc"] > 2:
            return "Fucosylated"
        elif d["HexNAc"] == 2:
            if "Hex" in d:
                if d["Hex"] > 4:
                    return "Complex"
            else:
                return "Paucimannose"
    elif d["HexNAc"] > 2:
        return "Complex"
    elif d["HexNAc"] <= 2:
        if "Hex" in d:
            if d["Hex"] <= 9 and d["Hex"] > 4:
                return "High Mannose"

        return "Paucimannose"


peptides.loc[:, "glycan_type"] = peptides.glycan.map(categorize_glycan)


In [25]:
# map unique glycopeptide id

peptides.loc[:, "pep_mods"] = peptides.apply(
    lambda x: x["sequence"] + "_" + x["modifications"], axis=1
)
peptides.head(3)


,accession,description,checked,confidence,annotated_sequence,modifications,num_protein_groups,num_proteins,num_psms,master_protein_accessions,...,sequence_in_protein,positions_in_proteins,sequence,data_source,concentration,temperature,run,glycan,glycan_type,pep_mods
2,P61823,Ribonuclease pancreatic OS=Bos taurus OX=9913 ...,True,Medium,[R].NLTK.[D],1xHexNAc(2) [N1],1,1,1,P61823,...,R.NLTK.D,[60-63],NLTK,00625_30C_Run1,0.0625,30C,Run1,HexNAc(2),Paucimannose,NLTK_1xHexNAc(2) [N1]
3,P61823,Ribonuclease pancreatic OS=Bos taurus OX=9913 ...,True,High,[R].NLTK.[D],1xHexNAc(2)Hex(1) [N1],1,1,4,P61823,...,R.NLTK.D,[60-63],NLTK,00625_30C_Run1,0.0625,30C,Run1,HexNAc(2)Hex(1),Paucimannose,NLTK_1xHexNAc(2)Hex(1) [N1]
4,P61823,Ribonuclease pancreatic OS=Bos taurus OX=9913 ...,True,High,[R].NLTK.[D],1xHexNAc(2)Hex(2) [N1],1,1,4,P61823,...,R.NLTK.D,[60-63],NLTK,00625_30C_Run1,0.0625,30C,Run1,HexNAc(2)Hex(2),Paucimannose,NLTK_1xHexNAc(2)Hex(2) [N1]


In [26]:
peptides = peptides.reset_index()

# What are we even doing?

`Intact Glycopeptide Analysis at Elevated Temperatures Reveals limitations and Considerations in Label Free Quantitation`

1. We know that different temperatures enable different levels of performance in glycopeptide identificatioS
- show the difference between temperatures (num glycopeptideP)
- pinopint what is responsible. Increased peak area? Peak inte Is the separation of isomers somehow causing identification errors?nsity?

2. Is this a global occurrence or are various glycan types affected differently?
- For each temperature and glycan types, show the relative identification rates
- Using the pinpointed cause of different performance from aim1, show positive and negative examples for glycan types.

3. How does this effect quantitation?
- Is LFQ even remotely possible when temperature is varied?
- Is the ability to do LFQ rescued when analyzing at one temperature?
- Can we achieve greater LOQ when running at the optimal temperature?
- Are there any glycan types that show no ability to do LFQ?
- Is there a method of LFQ that is more useful than the other?

## 1a

Taking the max concentration, what is the difference between temperature?

In [27]:
# information to extract: num peptides, num unique glycans, num unique peptide backbones
temps, vals, kind, concent = [], [], [], []

for conc in peptides.concentration.unique():
    head = peptides[peptides.concentration == conc]

    for temp in head.temperature.unique():
        body = head[head.temperature==temp]

        for run in body.run.unique():
            tail = body[body.run == run]

            unique_gly = len(tail.pep_mods.unique())
            concent.append(conc)
            vals.append(unique_gly)
            temps.append(temp)
            kind.append("Unique Glycopeptides")

            unique_seq = len(tail.sequence.unique())
            concent.append(conc)
            vals.append(unique_seq)
            temps.append(temp)
            kind.append("Unique Backbones")

            unique_glycans = len(tail.glycan.unique())
            concent.append(conc)
            vals.append(unique_glycans)
            temps.append(temp)
            kind.append("Unique Glycans")

res = pd.DataFrame({"Temperature": temps, "Kind": kind, "Values": vals, "Concentration": concent})

my_colors = alt.Color(
    "Temperature:N",
    scale=alt.Scale(
        domain=["30C", "45C", "60C"], range=["#6E6581", "#B0B2BB", "#6B8A97"]
    ),
)

conc_max_comparison = (
    alt.Chart(res)
    .mark_bar()
    .encode(
        x=alt.X("Temperature:N", axis=alt.Axis(labelAngle=-45)),
        y=alt.Y("mean(Values):Q", title="Number of Identifications"),
        color=my_colors
    )
).properties(
    width=75,
    height=75
)

cmc_err_bars = (
    alt.Chart(res)
    .mark_errorbar(extent='stdev')
    .encode(
        x=alt.X("Temperature:N", title="", axis=alt.Axis(labelAngle=-45)),
        y=alt.Y("mean(Values):Q", title=""),
    )
)

base = alt.layer(conc_max_comparison, cmc_err_bars).facet(
    column="Kind:N", spacing=50,
)

max_conc = base.transform_filter(
        alt.datum.Concentration == 1.0
    ).properties(
        title='Temperature Differences, Max Concentration'
    )


all_conc = alt.vconcat()
for concentration in res.Concentration.unique():
    all_conc &= base.transform_filter(alt.datum.Concentration==concentration).properties(title=f'Concentration = {concentration}')


In [28]:
all_conc

alt.VConcatChart(...)

# Observations
When comparing across temperatures, there is often a slight difference in number of identifications between 30C and 45C. However, this difference is small and shows no consistent trend between concentrations analyzed.

However, at all concentrations tested, their is a significant difference when running at 60C. What could be causing this?



## 1b

What is a main source of this difference?

## 2a

Do all glycan types experience this trend?

In [60]:
glycan_sub = peptides[
    [
        "sequence",
        "glycan",
        "glycan_type",
        "pep_mods",
        "temperature",
        "concentration",
        "run",
    ]
]

base = (
    alt.Chart(glycan_sub)
    .mark_bar()
    .encode(
        x=alt.X("temperature:N", title="", axis=alt.Axis(labelAngle=-45)),
        y=alt.Y("count(glycan_type):N", title="Number Glycan Matches"),
        column="glycan_type:N",
        color="glycan_type:N",
    )
)

chart = alt.vconcat()
for concentration in glycan_sub.concentration.unique():
    chart &= base.transform_filter(alt.datum.concentration == concentration).properties(
        title=f"Concentration = {concentration}"
    )
chart

alt.VConcatChart(...)

In [55]:
glycan_sub[glycan_sub.glycan_type.isna()]

,sequence,glycan,glycan_type,pep_mods,temperature,concentration,run
2116,NLTKDRCKPVNTFVHESLADVQAVCSQK,HexNAc(2)Fuc(1),None,NLTKDRCKPVNTFVHESLADVQAVCSQK_1xAcetyl [N-Term]...,45C,0.5,Run1
2376,TFMLAASWNGTK,HexNAc(2)Fuc(1),None,TFMLAASWNGTK_1xHexNAc(2)Fuc(1) [N9],60C,0.5,Run1
